## Uruchomienie wieloetapowego systemu rekomendacji na Triton Inference Server

Na podstawie zapytania zawierającego jedynie identyfikator uzytkownika nastąpi rekomendacja id produktó.

### Import bibliotek i funkcji

In [2]:
import os
import numpy as np
import pandas as pd
import feast
import faiss
import seedir as sd
from nvtabular import ColumnSchema, Schema

from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.session_filter import FilterCandidates
from merlin.systems.dag.ops.softmax_sampling import SoftmaxSampling
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ops.unroll_features import UnrollFeatures
from merlin.systems.triton.utils import run_triton_server, run_ensemble_on_tritonserver

/usr/local/lib/python3.8/dist-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/usr/local/lib/python3.8/dist-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
11/22/2022 07:34:21 PM INFO:Loading faiss with AVX2 support.
11/22/2022 07:34:21 PM INFO:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
11/22/2022 07:34:21 PM INFO:Loading faiss.
11/22/2022 07:34:21 PM INFO:Successfully loaded faiss.
/usr/local/lib/python3.8/dist-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


### Umieszczenie atrybutów w repozytorium atrybutw

Repozytorium atrybutów [Feast](https://docs.feast.dev/getting-started/architecture-and-components/registry) przechowuje wszelkie atrybuty produktów oraz użytkowników na potrzeby wytwarzania (treningu) modeli jak i ich produkcyjnego wykorzystania (inferencji). Atrybuty użytkowników oraz produktów zostały programistycznie opisane w plikach `user_features.py` oraz  `item_features.py` files. Za pomocą klasy FeatureView() można umieścić dane z wielu źródeł danych w Feast i wykorzystać je do treningu lub inferencji. W plikach `user_features.py` oraz `item_features.py` znajdują się informacje gdzie Feast ma sięgać po atrybuty uzytkowników i produktów.

Instrukcja `feast apply` wykonana poniżej na podstawie wspomnianych plików tworzy lokalną bazę danych SQLite `online_store.db` zawierającą wszelkie atrybuty (tzw. repozytorium offline).

In [5]:
BASE_DIR = os.environ.get("BASE_DIR", "/Merlin/examples/Building-and-deploying-multi-stage-RecSys/")

# define feature repo path
feast_repo_path = BASE_DIR + "feature_repo/"

In [3]:
%cd $feast_repo_path
!feast apply

/Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo
/usr/local/lib/python3.8/dist-packages/cupy/_environment.py:439: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda116, cupy-cuda11x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
/usr/local/lib/python3.8/dist-packages/feast/feature_

### Ładowanie atrybutów do repozytorium online

Po wykonaniu instrukcji `apply` należy przeprowadzić tzw. operację [materializacji](https://docs.feast.dev/how-to-guides/running-feast-in-production) w wyniku której otrzymmay repozytorium atrybutów online (szybkiego dostępu). Szybki dostęp do tych danych jest niezbędny dla dostarczania rekomendacji z niskim opóźnieniem.


In [4]:
!feast materialize 1995-01-01T01:01:01 2025-01-01T01:01:01

/usr/local/lib/python3.8/dist-packages/cupy/_environment.py:439: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda116, cupy-cuda11x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
Materializing 2 feature views from 1995-01-01 01:01:01+00:00 to 2025-01-01 01:01:01+00:00 into the sqlite online store.

item

Otrzymana struktura katalogu feature_repo po instrukcjach `apply` oraz `materialize`.

In [5]:
# set up the base dir to for feature store
feature_repo_path = os.path.join(BASE_DIR, 'feature_repo')
sd.seedir(feature_repo_path, style='lines', itemlimit=10, depthlimit=5, exclude_folders=['.ipynb_checkpoints', '__pycache__'], sort=True)

feature_repo/
├─__init__.py
├─data/
│ ├─item_features.parquet
│ ├─online_store.db
│ ├─registry.db
│ └─user_features.parquet
├─feature_store.yaml
├─item_features.py
└─user_features.py


### Przeliczenie indeksu Faiss index, klient repozytorium atrybutów

Faiss jest silnikiem pozwalającym na znalezienie "bliskich" wektorów reprezentacji prduktów za pomocą algorytmu ANN, do działania potrzebuje utwaorzyć wewnętrzne indeksy:

In [6]:
if not os.path.isdir(os.path.join(BASE_DIR + 'faiss_index')):
    os.makedirs(os.path.join(BASE_DIR + 'faiss_index'))

Ustawianie ścieżek dla modeli oraz indeksu Faiss:

In [6]:
faiss_index_path = BASE_DIR + 'faiss_index' + "/index.faiss"
retrieval_model_path = BASE_DIR + "query_tower/"
ranking_model_path = BASE_DIR + "dlrm/"

Schema żądania dla Triton Infrence Server (TIS) - czyli w jakim formacie przekazać żądanie do serwera o rekomendację dla konkretnego użytkownika opisanego identyfikatorem user_id.

In [7]:
request_schema = Schema(
    [
        ColumnSchema("user_id", dtype=np.int32),
    ]
)

Operator `QueryFaiss` tworzy interfejs pomiędzy FAISS Approximate Nearest Neighbors (ANN) Index i Triton Infrence Server. Dla wejściowego wektora reprezentacji produktu, za pomocą algorytmu ANN znajdowane są k-najbliższe wektory z indeksu.

`setup_faiss` tworzy indeks Faiss na podstawie odległości pomiędzy wektorami reprezentacji (L2).

In [9]:
from merlin.systems.dag.ops.faiss import QueryFaiss, setup_faiss 

item_embeddings = np.ascontiguousarray(
    pd.read_parquet(BASE_DIR + "item_embeddings.parquet").to_numpy()
)
setup_faiss(item_embeddings, faiss_index_path)

Utworzenie klenta repozytorium atrybutów

In [8]:
feature_store = feast.FeatureStore(feast_repo_path)

Pobieranie atrybutów użytkownika z repozytorium za pomocą operatora `QueryFeast`.

In [9]:
from merlin.systems.dag.ops.feast import QueryFeast 

user_features = ["user_id"] >> QueryFeast.from_feature_view(
    store=feature_store,
    view="user_features",
    column="user_id",
    include_id=True,
)

/usr/local/lib/python3.8/dist-packages/merlin/systems/dag/ops/feast.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ValueType.FLOAT: (np.float, False, False),


Pobieranie kandydatów do rekomendacji za pomocą `retrieval model` dla konkretnego uzytkownika. Operator `PredictTensorflow()` przeszktałca model głębokiej sieci neuronowej stworzony w Tensorflow do formatu odpowiedniego dla TIS.

In [10]:
# konfiguracja Tensorflow tak aby nie alokował całej dostępnej pamięci GPu
from merlin.models.loader.tf_utils import configure_tensorflow

configure_tensorflow()

2022-11-22 19:36:54.067392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 19:36:54.067825: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 19:36:54.100054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 19:36:54.100298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 19:36:54.100473: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

<function tensorflow.python.dlpack.dlpack.from_dlpack(dlcapsule)>

In [12]:
from merlin.systems.dag.ops.faiss import QueryFaiss

topk_retrieval = 100
retrieval = (
    user_features
    >> PredictTensorflow(retrieval_model_path)
    >> QueryFaiss(faiss_index_path, topk=topk_retrieval)
)

2022-11-22 19:37:40.373395: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1077347120 exceeds 10% of free system memory.
11/22/2022 07:37:40 PM WARNING:No training configuration found in save file, so the model was *not* compiled. Compile it manually.


Pobieranie atrybutów dla kandydatów do rekomendacji z etapu retrieval z repozytorium atrybutów.

In [13]:
item_features = retrieval["candidate_ids"] >> QueryFeast.from_feature_view(
    store=feature_store,
    view="item_features",
    column="candidate_ids",
    output_prefix="item",
    include_id=True,
)

Merge atrybutów użytkownika i produktu na potrzeby etapu scoringu za pomocą operatora `UnrollFeatures`.

In [14]:
user_features_to_unroll = [
    "user_id",
    "user_shops",
    "user_profile",
    "user_group",
    "user_gender",
    "user_age",
    "user_consumption_2",
    "user_is_occupied",
    "user_geography",
    "user_intentions",
    "user_brands",
    "user_categories",
]

combined_features = item_features >> UnrollFeatures(
    "item_id", user_features[user_features_to_unroll]
)

Scoring na podstawie wszystkich atrybutów  za pomocą modelu DLRM (ranking).

In [15]:
ranking = combined_features >> PredictTensorflow(ranking_model_path)

2022-11-22 19:37:52.237459: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 811179008 exceeds 10% of free system memory.
2022-11-22 19:37:52.237716: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 811179008 exceeds 10% of free system memory.
2022-11-22 19:37:52.284843: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 811179008 exceeds 10% of free system memory.


Otrzymanie top 10 rekomendacji za pomocą operatora `SoftmaxSampling()`, to przykład aplikacji logiki biznesowej etapu ordering.

In [16]:
top_k=10
ordering = combined_features["item_id"] >> SoftmaxSampling(
    relevance_col=ranking["output_1"], topk=top_k, temperature=20.0
)

### Eksport zespołu rekomendacji (Ensemble)


In [18]:
if not os.path.isdir(os.path.join(BASE_DIR + 'poc_ensemble')):
    os.makedirs(os.path.join(BASE_DIR + 'poc_ensemble'))

In [17]:
# define the path where all the models and config files exported to
export_path = os.path.join(BASE_DIR + 'poc_ensemble')

ensemble = Ensemble(ordering, request_schema)

In [ ]:
ens_config, node_configs = ensemble.export(export_path)

Struktura katalogu zespołu rek.

In [20]:
sd.seedir(export_path, style='lines', itemlimit=10, depthlimit=5, exclude_folders=['.ipynb_checkpoints', '__pycache__'], sort=True)

poc_ensemble/
├─0_queryfeast/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
├─1_predicttensorflow/
│ ├─1/
│ │ └─model.savedmodel/
│ │   ├─assets/
│ │   ├─keras_metadata.pb
│ │   ├─saved_model.pb
│ │   └─variables/
│ │     ├─variables.data-00000-of-00001
│ │     └─variables.index
│ └─config.pbtxt
├─2_queryfaiss/
│ ├─1/
│ │ ├─index.faiss/
│ │ │ └─index.faiss
│ │ └─model.py
│ └─config.pbtxt
├─3_queryfeast/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
├─4_unrollfeatures/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
├─5_predicttensorflow/
│ ├─1/
│ │ └─model.savedmodel/
│ │   ├─assets/
│ │   ├─keras_metadata.pb
│ │   ├─saved_model.pb
│ │   └─variables/
│ │     ├─variables.data-00000-of-00001
│ │     └─variables.index
│ └─config.pbtxt
├─6_softmaxsampling/
│ ├─1/
│ │ └─model.py
│ └─config.pbtxt
└─ensemble_model/
  ├─1/
  └─config.pbtxt


In [21]:
export_path

'/Merlin/examples/Building-and-deploying-multi-stage-RecSys/poc_ensemble'

### Uruchamianie Triton Server

Za pomocą komendy:

```
tritonserver --model-repository=/ensemble_export_path/ --backend-config=tensorflow,version=2
```


### Otrzymywanie rekomendacji

In [ ]:

ensemble.graph.output_schema.column_names

In [20]:
from merlin.systems.triton.utils import send_triton_request
from merlin.core.dispatch import make_df

# create a request to be sent to TIS
request = make_df({"user_id": [1]})
request["user_id"] = request["user_id"].astype(np.int32)

# outputs = ensemble.graph.output_schema.column_names
outputs = ['ordered_ids']

In [21]:
response = send_triton_request(request, outputs)
response

{'ordered_ids': array([[2209489],
        [1921832],
        [1280386],
        [2763866],
        [ 863284],
        [1225053],
        [1928709],
        [ 983790],
        [ 102171],
        [1189684]], dtype=int32)}